In [2]:
import requests
import pandas as pd

def get_token_related_pools(token: str, chain: str = "Ethereum") -> pd.DataFrame:
    """
    回傳在指定鏈（預設 Ethereum）中，與指定 token（例如 'ENA'）相關的池子列表。
    """
    url = "https://yields.llama.fi/pools"
    res = requests.get(url)
    res.raise_for_status()
    data = res.json()["data"]

    # 過濾條件：在指定鏈上，symbol 用 "-" 拆分後有 token
    filtered_pools = [
        {
            "project": p["project"],
            "symbol": p["symbol"],
            "pool_id": p["pool"],
            "apy": p["apy"],
            "tvlUsd": p["tvlUsd"]
        }
        for p in data
        if p["chain"] == chain and token.upper() in p["symbol"].split("-")
    ]

    return pd.DataFrame(filtered_pools).sort_values(by='apy',ascending=False).head(1)

# ✅ 查詢 ENA 池子
df_ena = get_token_related_pools("ENA")
df_ena.head()

,project,symbol,pool_id,apy,tvlUsd
1,uniswap-v3,ENA-WETH,5a7ea2ca-8651-487e-90ca-e7b06102a050,285.69848,3367337


In [6]:
import requests
import pandas as pd

# 根據池子id返回apy資料
def get_yield_history(pool_id: str) -> pd.DataFrame:
    url = f"https://yields.llama.fi/chart/{pool_id}"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()["data"]

    df = pd.DataFrame(data)
    df["timestamp"] = pd.to_datetime(df["timestamp"])  # ⬅️ 移除 unit="s"
    return df[["timestamp","apy"]]

# 測試 Lido stETH 的 pool_id
df_steth = get_yield_history(df_ena['pool_id'].values[0])
df_steth


,timestamp,apy
0,2024-04-03 23:06:32.849000+00:00,1200.77102
1,2024-04-04 22:07:48.696000+00:00,707.25565
2,2024-04-05 23:07:14.093000+00:00,513.79050
3,2024-04-06 23:08:06.703000+00:00,306.27657
4,2024-04-07 23:06:47.143000+00:00,315.84091
...,...,...
473,2025-07-22 23:11:22.751000+00:00,156.57793
474,2025-07-23 23:13:24.703000+00:00,138.65753
475,2025-07-24 23:31:43.745000+00:00,256.00298
476,2025-07-25 23:13:12.213000+00:00,474.24318
